In [12]:
from hydra import initialize, compose
from pathlib import Path
import pandas as pd

In [13]:
with initialize(version_base=None, config_path="./conf/filepaths"):
    cfg = compose(config_name="filepaths")

In [16]:
df = pd.read_parquet(Path(cfg.interim_data) / "mapped_reactions.parquet")
print(len(df))
df.head(20)

16


,id,smarts,am_smarts,rule,reaction_center
0,0,NC(CCC=O)C(=O)O.NC(=O)c1ccc[n+](C2OC(COP(=O)(O...,[CH:1]([CH2:3][CH2:5][CH:6]([NH2:7])[C:8](=[O:...,[#6:1].[#6:2]1:[#6:3]:[#6:4]:[#6:5]:[#7+:6]:[#...,"[[[4], [3, 4, 5, 6, 7, 47], [2]], [[0, 1], [0,..."
1,1,CC(O)C(O)C(O)C(O)C(=O)O>>CC(O)C(O)CC(=O)C(=O)O.O,[CH:1]([CH:2]([OH:3])[C:9](=[O:10])[OH:11])([C...,([#6:1]-[#8:2].[#6:3]-[#8:4])>>([#6:1].[#6:3]=...,"[[[5, 6, 7, 8]], [[0, 1, 2], [0]]]"
2,3,CSCCC(=O)C(=O)O.O=CO>>CSCCC(=O)C(O)=CO.O=O,[C:1]([C:3]([CH2:5][CH2:7][S:8][CH3:9])=[O:6])...,[#6:1]=[#8:2].[#6:3]=[#8:4]>>[#6:1]=[#6:3].[#8...,"[[[6, 7], [1, 0]], [[0, 1], [0, 1]]]"
3,5,Nc1nc2c(ncn2C2OC(COP(=O)(O)OP(=O)(O)OC3OC(CO)C...,[CH:1]1([O:50][P:51]([O:52][P:55]([O:56][CH2:5...,([#8:1].[#6:2]-[#8:3]).[#6:4]-[#8:5]>>([#6:2]-...,"[[[31, 21, 20], [27, 26]], [[0, 1, 2, 3], [0]]]"
4,5,O=P(O)(OC1C(O)C(O)C(O)C(O)C1O)OC1C(O)C(O)C(O)C...,[CH:1]1([OH:50])[CH:3]([OH:6])[CH:5]([OH:10])[...,[#6:1]-[#8:2].[#8:3]-[#15:4]>>[#6:1]-[#8:3].[#...,"[[[36, 37], [20, 17]], [[0, 1], [0, 1]]]"
5,5,Nc1nc2c(ncn2C2OC(COP(=O)(O)OP(=O)(O)OC3OC(CO)C...,[CH:1]1([O:50][P:51]([O:52][P:55]([O:56][CH2:5...,[#6:1]-[#8:2].[#8:3]>>[#6:1]-[#8:3].[#8:2],"[[[21, 20], [37]], [[0, 1], [0]]]"
6,6,*C(=O)NC(*)C(=O)OC1C(COP(*)(=O)O)OC(n2cnc3c(N)...,[C:1]([CH:3]([NH:5][C:7]([*:8])=[O:9])[*:6])(=...,[#6:1]-[#8:2].[#8:3]>>[#6:1]-[#8:3].[#8:2],"[[[6, 8], [0]], [[0, 1], [0]]]"
7,6,*C(=O)NC(*)C(=O)OC1C(COP(*)(=O)O)OC(n2cnc3c(N)...,[O:1]([CH:4]1[CH:2]([OH:25])[CH:9]([n:11]2[cH:...,([#6:1]-[#8:2].[#6:3]-[#8:4]).[#8:5]>>([#8:2]....,"[[[6, 8, 29, 30], [0]], [[0, 1, 2], [0, 1]]]"
8,7,O=C(O)C1=CC(OP(=O)(O)O)C(O)C(O)C1.C=C(OP(=O)(O...,[CH:1]1([O:22][P:23](=[O:24])([OH:25])[OH:26])...,([#8:1].[#6:2]-[#8:3]).[#6:4]-[#8:5]>>([#6:2]-...,"[[[14, 5, 6], [1, 2]], [[0, 1, 2, 3], [0]]]"
9,7,O=C(O)C1=CC(OP(=O)(O)O)C(O)C(O)C1.C=C(OP(=O)(O...,[CH:1]1([OH:22])[CH:3]([OH:6])[CH:5]([O:9][P:1...,[#6:1]-[#8:2].[#8:3]-[#15:4]>>[#6:1]-[#8:3].[#...,"[[[13, 14], [2, 3]], [[0, 1], [0, 1]]]"
